In [1]:
!pip install torch

In [2]:
import numpy as np

In [3]:
data = np.load('../data/dataset.npz', allow_pickle=True)

X_train = data['arr_0']  # pixel data
Y_train = data['arr_1']  # labels
X_test = data['arr_2']   # pixel data
Y_test = data['arr_3']   # labels

In [4]:
num_classes = 46
train_per_class = 1600
val_per_class = 100

X_train_new, Y_train_new = [], []
X_val, Y_val = [], []

for i in range(num_classes):
    start = i * 1700
    end = start + 1700
    
    # Training: first 1600 images of this class
    for j in range(train_per_class):
        X_train_new.append(X_train[start + j])
        Y_train_new.append(Y_train[start + j])
    
    # Validation: last 100 images of this class
    for j in range(train_per_class, train_per_class + val_per_class):
        X_val.append(X_train[start + j])
        Y_val.append(Y_train[start + j])

# Convert to NumPy arrays (each element still keeps its original shape)
X_train_new = np.array(X_train_new)
Y_train_new = np.array(Y_train_new)
X_val = np.array(X_val)
Y_val = np.array(Y_val)

In [12]:
import numpy as np
import torch

# 1. Load raw data
data = np.load('../data/dataset.npz', allow_pickle=True)
X_train_raw = data['arr_0']
Y_train_raw = data['arr_1']
X_test_raw  = data['arr_2']
Y_test_raw  = data['arr_3']

num_classes = 46
train_per_class = 1600
val_per_class = 100

# 2. Manual Split (Respecting Class Structure)
X_train_new, Y_train_new = [], []
X_val, Y_val = [], []

for i in range(num_classes):
    start = i * 1700
    # Training: first 1600
    X_train_new.extend(X_train_raw[start : start + train_per_class])
    Y_train_new.extend(Y_train_raw[start : start + train_per_class])
    # Validation: last 100
    X_val.extend(X_train_raw[start + train_per_class : start + 1700])
    Y_val.extend(Y_train_raw[start + train_per_class : start + 1700])

# 3. Convert to Tensors & Normalize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_tensor(data_list, labels):
    x = torch.from_numpy(np.stack([img.flatten() for img in data_list])).float() / 255.0
    y = torch.tensor(labels, dtype=torch.long) - 1 # 0-indexed
    return x, y

X_train_t, Y_train_t = to_tensor(X_train_new, Y_train_new)
X_val_t, Y_val_t     = to_tensor(X_val, Y_val)
X_test_t, Y_test_t   = to_tensor(X_test_raw, Y_test_raw)

# 4. PCA Projection (Apply on GPU)
mean = X_train_t.mean(dim=0).to(device)
U, S, V = torch.pca_lowrank((X_train_t.to(device) - mean), q=200)

X_train_pca = torch.mm(X_train_t.to(device) - mean, V)
X_val_pca   = torch.mm(X_val_t.to(device) - mean, V)
X_test_pca  = torch.mm(X_test_t.to(device) - mean, V)

print(f"Data ready. Training shape: {X_train_pca.shape}")
print(f"Data ready. Val shape: {X_val_pca.shape}")
print(f"Data ready. Test shape: {X_test_pca.shape}")

Data ready. Training shape: torch.Size([73600, 200])
Data ready. Val shape: torch.Size([4600, 200])
Data ready. Test shape: torch.Size([13800, 200])


In [13]:
class DevnagariSVM:
    def __init__(self, num_features, n_components=1500, gamma=0.05):
        self.gamma = gamma
        self.n_components = n_components
        self.w = None
        self.b = None
        self.basis_points = None

    def fit_nystrom(self, X):
        # Pick anchor points from training data
        idx = torch.randperm(X.size(0))[:self.n_components]
        self.basis_points = X[idx].clone()

    def transform(self, X):
        # RBF Kernel Map: exp(-gamma * ||dist||^2)
        dist_sq = torch.cdist(X, self.basis_points).pow(2)
        K_base = torch.exp(-self.gamma * dist_sq)
        
        dist_basis = torch.cdist(self.basis_points, self.basis_points).pow(2)
        K_basis = torch.exp(-self.gamma * dist_basis)
        
        L, Q = torch.linalg.eigh(K_basis)
        L_inv_sqrt = torch.diag(1.0 / torch.sqrt(L + 1e-7))
        mapping_matrix = Q @ L_inv_sqrt @ Q.t()
        
        return K_base @ mapping_matrix

    def predict(self, X_pca_tensor):
        X_rbf = self.transform(X_pca_tensor)
        logits = X_rbf @ self.w - self.b
        return torch.argmax(logits, dim=1)

# Instantiate the machine
model = DevnagariSVM(num_features=200, n_components=1500, gamma=0.05)
model.fit_nystrom(X_train_pca)

In [15]:
from torch.utils.data import DataLoader, TensorDataset

# Transform training and validation to RBF space once to save time
X_train_rbf = model.transform(X_train_pca)
X_val_rbf   = model.transform(X_val_pca)

# Init Weights
num_rbf_features = X_train_rbf.shape[1]
model.w = torch.randn(num_rbf_features, 46, device=device, requires_grad=True)
with torch.no_grad(): model.w.mul_(0.01)
model.b = torch.zeros(46, device=device, requires_grad=True)

optimizer = torch.optim.Adam([model.w, model.b], lr=1e-4)
train_loader = DataLoader(TensorDataset(X_train_rbf, Y_train_t.to(device)), batch_size=1024, shuffle=True)

print("Starting Training...")
for epoch in range(1, 21):
    model.w.requires_grad = True
    model.b.requires_grad = True
    
    for b_x, b_y in train_loader:
        optimizer.zero_grad()
        scores = b_x @ model.w - model.b
        Y_onehot = torch.full_like(scores, -1.0).scatter_(1, b_y.unsqueeze(1), 1.0)
        
        margins = torch.clamp(1 - Y_onehot * scores, min=0)
        loss = 0.5 * torch.sum(model.w**2) + 1.0 * torch.sum(margins)
        loss.backward()
        optimizer.step()
    
    # Validation Check
    if epoch % 1 == 0:
        with torch.no_grad():
            preds = torch.argmax(X_val_rbf @ model.w - model.b, dim=1)
            val_acc = (preds == Y_val_t.to(device)).float().mean()
            print(f"Epoch {epoch} | Val Acc: {val_acc*100:.2f}%")

Starting Training...
Epoch 1 | Val Acc: 8.46%
Epoch 2 | Val Acc: 18.22%
Epoch 3 | Val Acc: 27.67%
Epoch 4 | Val Acc: 35.61%
Epoch 5 | Val Acc: 40.61%
Epoch 6 | Val Acc: 44.48%
Epoch 7 | Val Acc: 47.17%
Epoch 8 | Val Acc: 49.22%
Epoch 9 | Val Acc: 50.15%
Epoch 10 | Val Acc: 51.33%
Epoch 11 | Val Acc: 51.89%
Epoch 12 | Val Acc: 52.33%
Epoch 13 | Val Acc: 52.48%
Epoch 14 | Val Acc: 52.67%
Epoch 15 | Val Acc: 52.65%
Epoch 16 | Val Acc: 52.67%
Epoch 17 | Val Acc: 52.83%
Epoch 18 | Val Acc: 52.83%
Epoch 19 | Val Acc: 52.89%
Epoch 20 | Val Acc: 52.65%


In [16]:
with torch.no_grad():
    # Use the predict method (it handles the RBF transformation)
    test_preds = model.predict(X_test_pca)
    
    # Convert predictions back to 1-indexed for final comparison
    final_preds = (test_preds + 1).cpu().numpy()
    final_actual = Y_test_raw # The original 1-indexed labels
    
    test_acc = (final_preds == final_actual).mean()
    print(f"--- FINAL EVALUATION ---")
    print(f"Test Accuracy: {test_acc * 100:.2f}%")

--- FINAL EVALUATION ---
Test Accuracy: 53.52%
